# Revisiting Food-Safety Inspections from the Chicago Dataset - A Tutorial (Part 2)

## 1. In Review

## 2. Kernel Density Estimation
[HAVEN'T WRITTEN THE CODE FOR THIS]
* gaussian kde
* 90 day window
* kde for garbage, crime, sanitation
* call something other than heat map to avoid confuion?

## 3. Calculation of Model Data
We then set out derive all features originally considered by the Chicago team plus several novel features, translating from R to Python in the process. While relative weights in forecasting individual violations rather than number of critical violations may be different, we expect the significant databases and [NEEDS REPHRASING & STUFF]

[MAYBE HAVE BULLET POINTS FOR DATABASES, FEATURES]

Each row of the final training dataset represents an inspection, its outcomes, and a number of other features. [SOMETHING ABOUT TRAINING METHODOLOGY, MAYBE ABOUT FIGURING OUT WHICH FEATURES MATTER FOR WHICH]

We used the inspections database as the basis for our training data.

List of features is as follows:

### 3.1. features from food inspection data
[NEED TO WRITE THIS UP]
* used subset of inspections as basis
* merged with violation values & counts on inspection id
* created pass / fail flags
* sorted by date, shifted to get previous info
  * past fail
  * past critical, serious, minor
  * inspections with no past default to 0
* calculate time since last
  * binary first_record
  * time since last defaults to 2
  
### 3.2. features from business license data
[YET TO BE CALCULATED]
  
### 3.3. from heat maps
[YET TO BE CALCULATED]
  
### 3.4. from weather
[YET TO BE CALCULATED

## 4. Generating Model